### Install Layer

In [1]:
pip install layer-sdk

     |████████████████████████████████| 27.4 MB 101 kB/s 
     |████████████████████████████████| 2.9 MB 44.6 MB/s 
     |████████████████████████████████| 23.6 MB 73.2 MB/s 
     |████████████████████████████████| 271 kB 55.0 MB/s 
     |████████████████████████████████| 2.4 MB 63.2 MB/s 
     |████████████████████████████████| 212 kB 55.5 MB/s 
     |████████████████████████████████| 56 kB 4.7 MB/s 
     |████████████████████████████████| 159 kB 44.1 MB/s 
     |████████████████████████████████| 90 kB 9.5 MB/s 
     |████████████████████████████████| 16.9 MB 44 kB/s 
     |████████████████████████████████| 1.3 MB 59.1 MB/s 
     |████████████████████████████████| 374 kB 56.5 MB/s 
     |████████████████████████████████| 3.5 MB 46.0 MB/s 
     |████████████████████████████████| 131 kB 71.4 MB/s 
     |████████████████████████████████| 212.4 MB 63 kB/s 
     |████████████████████████████████| 596 kB 62.6 MB/s 
     |████████████████████████████████| 97 kB 6.9 MB/s 
     |██████████████

In [2]:
pip install -U ipython

     |████████████████████████████████| 788 kB 8.1 MB/s 
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.21 which is incompatible.
google-colab 1.0.0 requires ipython~=5.5.0, but you have ipython 7.28.0 which is incompatible.


In [1]:
import layer
layer.login()


Please open the following link in your web browser. Once logged in, copy the code and paste it here.
https://auth.beta.layer.co/authorize?response_type=code&code_challenge=Q0TT83W8cHot-t_sdhYwfNOeakaGUbKvbz_h96agYUc&code_challenge_method=S256&client_id=tvp1Wo8acTt4z46c7MbUB3Rg5MqfmqkN&redirect_uri=https://beta.layer.co/oauth/code&scope=offline_access&audience=https://beta.layer.co
Code: udUMcQn08Rlv6D6B
Successfully logged into https://beta.layer.co


### fetch the dataset

In [2]:
dataset = layer.get_dataset("titanic_dataset")
df = dataset.to_pandas()
df.head()

,SURVIVED,PASSENGERID,EMBARKED,NAME,AGE,SEX,FARE,PARCH,CABIN,TICKET,PCLASS,SIBSP
0,0,1,S,"Braund, Mr. Owen Harris",22.0,male,7.2500,0,None,A/5 21171,3,1
1,1,2,C,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,female,71.2833,0,C85,PC 17599,1,1
2,1,3,S,"Heikkinen, Miss. Laina",26.0,female,7.9250,0,None,STON/O2. 3101282,3,0
3,1,4,S,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,female,53.1000,0,C123,113803,1,1
4,0,5,S,"Allen, Mr. William Henry",35.0,male,8.0500,0,None,373450,3,0


### Fetch the calculated features as a Spark DataFrame

In [3]:
pf = layer.get_featureset("passenger_features_spark").to_spark()
pf.show()

+-----------+--------+-------------+---------+--------+---+--------+-----+
|PASSENGERID|AGE_BAND|EMBARK_STATUS|FARE_BAND|IS_ALONE|SEX|SURVIVED|TITLE|
+-----------+--------+-------------+---------+--------+---+--------+-----+
|          1|       1|            0|        0|       0|  1|       0|    1|
|          2|       2|            1|        3|       0|  0|       1|    3|
|          3|       1|            0|        1|       1|  0|       1|    2|
|          4|       2|            0|        3|       0|  0|       1|    3|
|          5|       2|            0|        1|       1|  1|       0|    1|
|          6|       1|            2|        1|       1|  1|       0|    1|
|          7|       3|            0|        3|       1|  1|       0|    1|
|          8|       0|            0|        2|       0|  1|       0|    4|
|          9|       1|            0|        1|       0|  0|       1|    3|
|         10|       0|            1|        2|       0|  0|       1|    3|
|         11|       0|   

### Explore the Model Catalog

In [4]:
model = layer.get_model("survival_model_spark")
print(model.parameters)
print(model.metrics)

{'seed': '42', 'test_size': '0.2'}
{'BinaryClassificationEvaluator': [(1635250815161, 0.876865671641791)]}


#### Make a prediction

In [5]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null 
!wget -q https://downloads.apache.org/spark/spark-2.4.8/spark-2.4.8-bin-hadoop2.7.tgz 
!tar xf spark-2.4.8-bin-hadoop2.7.tgz

In [6]:
import os 
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64" 
os.environ["SPARK_HOME"] = "/content/spark-2.4.8-bin-hadoop2.7"

In [7]:
pip install findspark

In [8]:
import findspark 
findspark.init()

In [10]:
import pyspark
sc = pyspark.SparkContext.getOrCreate();

In [11]:
from pyspark.sql import SQLContext


In [43]:
test = sqlContext.createDataFrame([
    (0, 0,1,1,0,1),
    ], ["EMBARK_STATUS", "FARE_BAND","SEX","AGE_BAND","IS_ALONE","TITLE"])

In [44]:
test.show()

+-------------+---------+---+--------+--------+-----+
|EMBARK_STATUS|FARE_BAND|SEX|AGE_BAND|IS_ALONE|TITLE|
+-------------+---------+---+--------+--------+-----+
|            0|        0|  1|       1|       0|    1|
+-------------+---------+---+--------+--------+-----+



In [42]:
from pyspark.ml.feature import VectorAssembler


In [46]:
feat_cols = ['AGE_BAND', 'EMBARK_STATUS', 'FARE_BAND', 'IS_ALONE', 'SEX', 'TITLE']
vec_assember = VectorAssembler(inputCols=feat_cols, outputCol='features')
test_data = vec_assember.transform(test)

In [47]:
trained_classifier = model.get_train()
trained_classifier

PipelineModel_d0c9db1acdf3

In [49]:
predictions = trained_classifier.transform(test_data)


In [50]:
predictions

DataFrame[EMBARK_STATUS: bigint, FARE_BAND: bigint, SEX: bigint, AGE_BAND: bigint, IS_ALONE: bigint, TITLE: bigint, features: vector, rawPrediction: vector, probability: vector, prediction: double]

In [52]:
predictions.select('probability','AGE_BAND','prediction','SEX','FARE_BAND','EMBARK_STATUS','IS_ALONE','TITLE').show()


+--------------------+--------+----------+---+---------+-------------+--------+-----+
|         probability|AGE_BAND|prediction|SEX|FARE_BAND|EMBARK_STATUS|IS_ALONE|TITLE|
+--------------------+--------+----------+---+---------+-------------+--------+-----+
|[0.94872612797609...|       1|       0.0|  1|        0|            0|       0|    1|
+--------------------+--------+----------+---+---------+-------------+--------+-----+

